In [5]:
import os, nltk
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
os.getcwd()

'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [6]:
def get_word2vec_embeddings(dataset:pd.DataFrame, verbose:bool=False, dim:int=200) :
    sentences = []
    for sentence in list(dataset["text"]) :
        sentences.append(nltk.word_tokenize(sentence))
    if verbose : 
        print("No. of Sentences:", len(sentences))
        for sentence in sentences[ : 5] : print("   ", sentence)
    
    model = Word2Vec(sentences=sentences, vector_size=dim, window=5, min_count=1, workers=4)
    model.train(sentences, epochs=100, total_examples=model.corpus_count, total_words=model.corpus_total_words, compute_loss=True)
    
    words = model.wv.index_to_key
    if verbose : 
        print("No. of Words:", len(words))
        for word in words[ : 5] : print("   ", word)
    
    vectors = []
    for word in words :
        vectors.append(model.wv[word])
    vectors = np.array(vectors)
    
    embeddings = pd.DataFrame(vectors, index=words, columns=range(1, dim + 1))
    
    return embeddings

In [8]:
# Para datasets com k_folding
# Args
k = 10
dataset_folder = "../resources/datasets/TwitterAirlines"
dataset_name = "TweetsProcessed2"
# dataset_folder = "../resources/datasets/IMDBMovieReviews"
# dataset_name = "IMDB_MRProcessed"

# Embeddings
save_path = f"{dataset_folder}/embeddings"
if not os.path.exists(save_path) : os.makedirs(save_path)

folds = [pd.read_csv(f"{dataset_folder}/folds/{dataset_name}_Fold{i_fold + 1}.csv", index_col=0) for i_fold in range(k)]

fold_embeddings = []
for i_fold in range(k) :
    trainset = pd.concat(folds[ : i_fold] + folds[i_fold + 1 : ])
    embeddings = get_word2vec_embeddings(trainset, dim=300)
    embeddings.to_csv(f"{save_path}/{dataset_name}_Fold{i_fold + 1}.csv")
    fold_embeddings.append(embeddings)
fold_embeddings[0]

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
to,0.516435,0.347733,-0.452792,0.088457,0.039299,-2.245621,1.647378,1.310579,0.678390,-0.790236,...,0.411603,0.290859,0.107042,-0.779314,0.259411,-0.152420,0.708290,0.060583,-0.517372,-1.263536
i,-0.219392,0.868007,-0.940412,-1.694158,1.072378,-1.477332,0.058441,0.849889,1.005164,-2.197675,...,1.270338,-0.148660,-1.975768,-0.748865,0.396474,0.917793,1.186704,-0.799262,0.320459,0.495669
the,0.812115,0.603371,-0.977580,-0.547784,-1.165419,-0.078007,0.598055,-0.043790,-0.164646,-1.004631,...,0.325372,0.545254,-0.082030,-0.105519,0.529428,-0.888825,0.611942,0.176268,0.943490,-0.552509
a,-0.253051,0.079646,-1.244305,-0.151374,-0.657647,-1.130011,0.555372,0.182149,1.623513,-1.580246,...,1.141599,-0.267230,-0.192988,0.521902,-0.074032,-0.591412,0.012265,0.805719,0.290775,-0.737753
you,-0.101054,-1.602474,0.512774,-0.871327,1.375609,-0.115770,-0.134071,0.939963,0.994703,0.127985,...,1.199927,0.329443,-1.701032,-0.476374,-0.388293,-0.936725,0.953041,0.859887,0.123127,0.069152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,-0.117521,0.105164,-0.230925,0.031505,-0.046003,-0.292806,0.014699,0.349486,0.215918,-0.007597,...,0.256132,-0.010253,0.069876,-0.095160,0.115071,0.206615,0.151744,-0.167131,0.171076,-0.194836
arrangement,0.132880,0.052602,0.177719,0.024279,0.189649,-0.400314,-0.137722,0.111640,-0.050551,0.070838,...,0.156535,0.319833,0.181603,0.028461,0.150362,0.120446,0.077672,-0.094794,-0.127063,-0.133468
cncled,0.146069,0.248842,-0.122057,-0.151805,0.027676,0.069226,-0.375591,0.218387,0.116919,0.159562,...,0.132486,0.417452,-0.026694,-0.003067,0.029017,0.125136,-0.216179,-0.022970,0.205853,0.280056
0223,0.207825,0.396298,-0.270329,-0.145979,0.077503,0.020073,-0.250553,0.187721,0.208813,0.066279,...,0.085990,0.277539,0.062296,0.027458,-0.034936,0.013833,-0.206100,-0.005330,0.326830,0.210818


In [4]:
# Para dataset simples
# Args
dataset_folder = "../resources/datasets/StanfordSentimentTreebank"
dataset_name = "SST2Processed2-train"
trainset = pd.read_csv(f"{dataset_folder}/split/{dataset_name}.csv")

# Exec
embeddings = get_word2vec_embeddings(trainset, dim=300)
embeddings.to_csv(f"{dataset_folder}/embeddings/{dataset_name}.csv")
embeddings

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
the,-0.654212,0.811368,0.384531,-0.899714,-1.484573,0.030267,0.174715,-0.843628,1.227235,-0.715294,...,-0.168738,0.358424,0.925911,0.024859,0.884300,0.073570,-0.098305,-1.388939,-0.206653,-0.733193
a,-0.835335,0.538750,0.289676,0.093590,-0.567916,-0.995128,-1.077931,-0.685834,0.883427,-0.629355,...,0.948870,0.506105,0.399434,-0.487153,0.616448,-0.455358,-0.182450,-1.630853,0.481335,-1.821020
and,-0.857282,-0.201412,-0.331601,0.095220,-0.593782,0.727717,-0.347292,-0.751720,0.428075,0.865392,...,1.528508,0.738855,0.868025,-0.673695,0.945312,-1.161142,0.525091,0.725009,-0.113804,-1.885472
of,-1.059606,0.979983,-0.352721,0.304755,-0.975741,-0.453731,-0.679021,0.208523,0.157430,-1.575512,...,1.560646,-1.820316,-0.651443,-1.223655,0.679445,-0.713050,-0.417852,-0.559250,-0.583354,-0.548991
to,-0.428921,0.571481,0.853159,-2.862955,0.066903,0.102770,-1.476636,0.717534,1.812182,-0.412397,...,1.983538,-1.718585,1.226137,0.228431,0.284531,-0.795667,-0.366460,-1.022310,-0.423707,0.389620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uproarious,0.242727,0.135954,-0.174169,0.103027,-0.090442,-0.244049,-0.100056,0.166321,-0.097042,-0.203938,...,0.066881,0.001746,0.319109,-0.156649,0.290942,0.184145,0.081197,0.190881,0.396384,-0.184348
millennium,-0.000830,-0.077660,0.119717,-0.062120,-0.010699,0.000233,0.001032,0.078888,-0.178524,-0.152518,...,-0.076265,0.241433,0.116743,0.011134,0.095797,0.122313,0.262005,0.144902,0.034403,-0.278402
1993,0.058110,-0.015617,-0.217982,0.071710,-0.160137,0.019209,0.092394,0.195716,0.057797,0.138385,...,0.123924,0.084932,0.073199,0.085665,-0.012736,0.144317,0.083628,0.115877,-0.094842,-0.024527
burden,-0.044068,0.207120,-0.031074,0.043188,-0.104619,-0.254110,0.180069,0.573262,-0.016899,-0.008746,...,0.166918,0.275306,0.053768,-0.086979,0.183824,0.020178,-0.021164,-0.079627,0.072991,0.005373
